## Preliminary operations

In [1]:
# import main libraries
import json
import requests
import pandas as pd
import numpy as np

In [2]:
# import all datasets
response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/incidentes2019.json")
incidents_db = json.loads(response.text)

response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/distancias_estaciones_barrios.json")
distances_stations_ntas_db = json.loads(response.text)

response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/distancias_estaciones.json")
distances_stations_db = json.loads(response.text)

response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/barrios.json")
ntas_db = json.loads(response.text)

response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/estaciones.json")
stations_db = json.loads(response.text)

In [3]:
# get a dataframe with the distances between stations and neighborhoods (ntas)
df_distance_stations_ntas = pd.read_csv("dataset/distance_stations_neighborhoods.csv", index_col="Unnamed: 0")
# get a dataframe with the distances between stations
df_distance_stations = pd.DataFrame(distances_stations_db)
# get number of ntas
n_nta = len(df_distance_stations_ntas)

In [4]:
# define a list with vehicle types
vehicle_types = ["engine", "ladder", "rescue", "squad", "hazardous"]
# define a dictionary with vehicle type quantities
vehicle_quantities = {
    "engine": 197, 
    "ladder": 143, 
    "rescue": 5, 
    "squad": 8, 
    "hazardous": 1
}

In [5]:
# create a dictionary to store the neighborhoods serviceability dataframes
dfs_nta_serviceability = {}
# create a dictionary to store the maximum servicability distance for each type of vehicle
serviceability_distances ={}

# iterate all vehicle types
for vehicle_type in vehicle_types:
    # create datframes of the same shape of the nta-station distances 
    dfs_nta_serviceability[vehicle_type] = df_distance_stations_ntas.copy()
    # set all the values in the serviceability dataframe to an initial value equal to True
    dfs_nta_serviceability[vehicle_type].loc[:] = True

    # set the initial serviceability distance to an initial value equal to 0
    serviceability_distances[vehicle_type] = 0

## Function *set_serviceability*

In [6]:
# define a function to set the serviceability of each nta from each station
def set_serviceability(distance_threshold, df_nta_serviceability):
    # interate nta-station distances dataframe
    for (nta, distances) in df_distance_stations_ntas.iterrows():
        # iterate stations' distances from the neighborhood
        for station, distance_st_nta in distances.items():
            # check if the distance between the nta and the station is greater then the threshold
            if distance_st_nta > distance_threshold:
                # set the neighborhood as unservable by that station
                df_nta_serviceability.loc[nta, station] = False
    
    # return the serviceability table ready
    return df_nta_serviceability

## Set the serviceability distance for each vehicle type

In [7]:
# get the serviceability distance for each type
for vehicle_type in vehicle_types:
    # get vehicle type quantity
    vehicle_quantity = vehicle_quantities[vehicle_type]
    # Set the number of neighborhoods you want to cover
    num_neighborhoods_to_cover = 2 if vehicle_quantity > n_nta else round((n_nta / vehicle_quantity) + 1)
    # Initialize an empty dictionary to store minimum thresholds for each station
    min_thresholds = {}

    # Iterate over each station in the DataFrame
    for station in df_distance_stations_ntas.columns:
        # Sort the neighborhoods by distance to the current station
        sorted_neighborhoods = df_distance_stations_ntas.sort_values(by=station)
        # Select the top nearest N neighborhoods to cover, where N is num_neighborhoods_to_cover
        selected_neighborhoods = sorted_neighborhoods.index[:num_neighborhoods_to_cover]
        # Calculate the maximum distance threshold for the selected neighborhoods
        max_threshold = sorted_neighborhoods.loc[selected_neighborhoods, station].max()
        # Store the result in the dictionary
        min_thresholds[station] = max_threshold

    # Find the maximum threshold among all stations
    max_threshold = max(min_thresholds.values())
    print(f"The maximum threshold to cover {num_neighborhoods_to_cover} neighborhoods is: {max_threshold}")

    # get a new copy of the original dataframe
    ntas_serviceability = dfs_nta_serviceability[vehicle_type].copy()
    # set the serviceability based on the last valid threshold (actual + 1)
    ntas_serviceability = set_serviceability(max_threshold, ntas_serviceability).copy()
    # store the serviceability table
    dfs_nta_serviceability[vehicle_type] = ntas_serviceability


The maximum threshold to cover 2 neighborhoods is: 800.24
The maximum threshold to cover 2 neighborhoods is: 800.24
The maximum threshold to cover 40 neighborhoods is: 2675.74
The maximum threshold to cover 25 neighborhoods is: 2339.26
The maximum threshold to cover 196 neighborhoods is: 4537.84


## Export data

In [8]:
# save the serviceability tables
for df in dfs_nta_serviceability:
    dfs_nta_serviceability[df].to_csv("dataset/serviceability/" + df + "_ntas_serviceability.csv")